## Data analysis of the billboard 100 data

This document is part of the showcase, where I replicate the same brief and simple analyses with different tools.

This particular file focuses on data analysis (a few queries) of the billboard 100 data from the tidytuesday project.

The data can be found in <https://github.com/rfordatascience/tidytuesday/tree/master/data/2021/2021-09-14>. They consist of two documents: *billboard.csv* contains information about the songs focusing on their position in the top100 list at different weeks. *audio_features.csv* contains information about specific attributes of the songs from spotify.

For the specific analysis I will use **Python** and **Dask** (plus **Jupyter notebook**).

We start by loading the packages:

In [1]:
from dask.distributed import Client
client = Client(n_workers=4)

and the *billboard* datset:

In [2]:
import dask.dataframe as dd
import pandas as pd
billboard = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-14/billboard.csv')
billboard = dd.from_pandas(billboard, npartitions=4)

We can have a look at information about the dask dataframe:

In [3]:
billboard.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 10 entries, url to weeks_on_chart
dtypes: object(5), float64(1), int64(4)

and the summary statistics:

In [4]:
billboard.describe().compute()

,week_position,instance,previous_week_position,peak_position,weeks_on_chart
count,327895.000000,327895.000000,295941.000000,327895.000000,327895.000000
mean,50.499309,1.072538,47.604066,41.358307,9.153793
std,28.865707,0.334188,28.056915,29.542497,7.590281
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,39.000000,1.000000,33.000000,19.000000,3.000000
50%,66.000000,1.000000,62.000000,45.000000,8.000000
75%,84.000000,1.000000,81.000000,82.000000,19.000000
max,100.000000,10.000000,100.000000,100.000000,87.000000


For the first main query, our aim is to select only the songs that have reached the No 1 spot of the billboard and see how many weeks they have stayed at the billboard in total (Note that sorting doesnot work for dask Series):

In [5]:
top_songs = billboard[billboard['peak_position'] == 1]
top_songs = top_songs.drop(['url', 'week_position', 'previous_week_position'], axis=1)
top_songs = top_songs.groupby(['performer', 'song']).weeks_on_chart.max()
top_songs.compute().reset_index(name="max_weeks")

,performer,song,max_weeks
0,50 Cent Featuring Olivia,Candy Shop,23
1,? (Question Mark) & The Mysterians,96 Tears,15
2,Ace Of Base,The Sign,41
3,Adele,Hello,26
4,Aerosmith,I Don't Want To Miss A Thing,20
...,...,...,...
1119,Walter Murphy & The Big Apple Band,A Fifth Of Beethoven,28
1120,Will Smith Featuring Dru Hill & Kool Mo Dee,Wild Wild West,17
1121,Wiz Khalifa,Black And Yellow,25
1122,XXXTENTACION,Sad!,38


Now we load the *audio features* dataset:

In [6]:
audio_features = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-14/audio_features.csv')
audio_features = dd.from_pandas(audio_features, npartitions=4)

And once again we look at the summary statistics:

In [7]:
audio_features.describe().compute()

,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
count,2.439700e+04,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24334.000000,24397.000000
mean,2.206843e+05,0.599945,0.618096,5.231651,-8.664607,0.727172,0.073554,0.294635,0.032539,0.192098,0.601746,120.276066,3.931577,41.224413
std,6.774671e+04,0.153133,0.199078,3.560211,3.601119,0.445422,0.083153,0.282300,0.136276,0.159073,0.238645,28.046937,0.320858,22.477405
min,2.968800e+04,0.000000,0.000581,0.000000,-28.030000,0.000000,0.000000,0.000003,0.000000,0.009670,0.000000,0.000000,0.000000,0.000000
25%,1.770295e+05,0.506000,0.487000,2.000000,-10.877750,0.000000,0.032500,0.051800,0.000000,0.091700,0.427000,100.264000,4.000000,25.000000
50%,2.160265e+05,0.613000,0.643000,5.000000,-7.981000,1.000000,0.042200,0.215000,0.000005,0.134000,0.628000,119.947000,4.000000,45.000000
75%,2.548132e+05,0.715000,0.781000,8.000000,-5.717000,1.000000,0.072400,0.539000,0.000510,0.255750,0.806000,137.515500,4.000000,61.000000
max,3.079157e+06,0.988000,0.997000,11.000000,2.291000,1.000000,0.951000,0.991000,0.982000,0.999000,0.991000,241.009000,5.000000,100.000000


For the second main query, our aim is to derive information about the peak position a song has reached in the billboard and the main spotify information. For this, we need to join the two datasets:

In [8]:
merge_left = billboard.groupby(['performer', 'song', 'song_id']).peak_position.max().compute().reset_index(name="best_position")
merge_right = audio_features[['song_id', 'performer', 'song', 'spotify_genre', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
data_join = dd.merge(merge_left, merge_right, on="song_id")
data_join.head(10)

,performer_x,song_x,song_id,best_position,performer_y,song_y,spotify_genre,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,"""Weird Al"" Yankovic",Amish Paradise,"Amish Paradise""Weird Al"" Yankovic",65,"""Weird Al"" Yankovic",Amish Paradise,"['antiviral pop', 'comedy rock', 'comic', 'par...",0.728,0.448,8.0,-10.540,0.1720,0.10300,0.000000,0.2670,0.483,80.902
1,"""Weird Al"" Yankovic",Canadian Idiot,"Canadian Idiot""Weird Al"" Yankovic",82,"""Weird Al"" Yankovic",Canadian Idiot,"['antiviral pop', 'comedy rock', 'comic', 'par...",0.543,0.697,8.0,-9.211,0.0612,0.00206,0.000002,0.3430,0.861,185.978
2,"""Weird Al"" Yankovic",Eat It,"Eat It""Weird Al"" Yankovic",59,"""Weird Al"" Yankovic",Eat It,"['antiviral pop', 'comedy rock', 'comic', 'par...",0.767,0.811,7.0,-8.548,0.0766,0.08660,0.000000,0.0684,0.858,147.423
3,'N Sync,(God Must Have Spent) A Little More Time On You,(God Must Have Spent) A Little More Time On Yo...,60,'N Sync,(God Must Have Spent) A Little More Time On You,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,'N Sync,Bye Bye Bye,Bye Bye Bye'N Sync,42,'N Sync,Bye Bye Bye,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,'Til Tuesday,(Believed You Were) Lucky,(Believed You Were) Lucky'Til Tuesday,98,'Til Tuesday,(Believed You Were) Lucky,"['boston rock', 'dance rock', 'new romantic', ...",0.612,0.523,5.0,-11.425,0.0321,0.44800,0.000002,0.0727,0.495,124.315
6,'Til Tuesday,Coming Up Close,Coming Up Close'Til Tuesday,90,'Til Tuesday,Coming Up Close,"['boston rock', 'dance rock', 'new romantic', ...",0.368,0.473,8.0,-14.943,0.0316,0.18100,0.000251,0.0665,0.273,80.186
7,0,Deacon Blues,Deacon Blues0,86,0,Deacon Blues,"['album rock', 'art rock', 'blues rock', 'clas...",0.751,0.572,0.0,-12.324,0.0388,0.48100,0.000532,0.1050,0.580,115.839
8,1 Of The Girls,Do Da What,Do Da What1 Of The Girls,91,1 Of The Girls,Do Da What,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"10,000 Maniacs",Because The Night,"Because The Night10,000 Maniacs",92,"10,000 Maniacs",Because The Night,"['alternative metal', 'christian rock', 'nu me...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
